# Collected data

In [ ]:
import pandas as pd

df = pd.read_csv('eval.csv')

print(df.to_markdown(index=False)) 

| Picture            | Object    | Predicted Box                             | GT Box                                    |   Predicted Length |   GT Length |   Predicted Width |   GT Width |
|:-------------------|:----------|:------------------------------------------|:------------------------------------------|-------------------:|------------:|------------------:|-----------:|
| Straight_no_shadow | Card      | [(230,181),(444,181),(445,314),(228,315)] | [(230,181),(444,181),(445,315),(228,315)] |                5.3 |         5.3 |               8.5 |        8.4 |
| Straight_no_shadow | Circle    | [(473,324),(575,340),(559,446),(455,430)] | [(473,324),(577,339),(561,449),(454,432)] |                4.4 |         3.7 |               3.9 |        3.7 |
| Straight_no_shadow | Complex   | [(244,381),(286,364),(310,428),(267,445)] | [(244,381),(282,366),(307,426),(269,443)] |                3.2 |         2.3 |               1.9 |        1.3 |
| Straight_no_shadow | Square    | [(562,238),(684,167),(755,282),(633,354)] | [(562,238),(684,167),(757,285),(634,357)] |                5.7 |         5   |               5.7 |        5   |
| Straight_no_shadow | Complex   | [(308,498),(796,473),(806,571),(315,596)] | [(312,497),(792,475),(805,593),(318,612)] |                3.7 |         4.1 |              19.5 |       19.3 |
| Tilted_cards       | Square    | [(278,145),(447,144),(442,284),(253,286)] | [(278,147),(447,144),(441,284),(256,287)] |                7.5 |         7.5 |               7.5 |        7.5 |
| Tilted_cards       | Card      | [(372,321),(586,368),(558,493),(326,438)] | [(372,324),(585,373),(558,493),(328,435)] |                8.7 |         8.7 |               5.1 |        5   |
| Tilted_cards       | Card      | [(621,146),(741,139),(776,304),(640,309)] | [(621,146),(741,139),(775,300),(639,305)] |                5.3 |         5.3 |               8.6 |        8.4 |

# Get Intersection over Union

In [7]:
pred_boxes = [
    [(230,181),(444,181),(445,314),(228,315)], 
    [(473,324),(575,340),(559,446),(455,430)], 
    [(244,381),(286,364),(310,428),(267,445)],
    [(562,238),(684,167),(755,282),(633,354)],
    [(308,498),(796,473),(806,571),(315,596)],
    [(278,145),(447,144),(442,284),(253,286)],
    [(372,321),(586,368),(558,493),(326,438)],
    [(621,146),(741,139),(776,304),(640,309)]
]

GT_boxes = [
    [(230,181),(444,181),(445,315),(228,315)],
    [(473,324),(577,339),(561,449),(454,432)],
    [(244,381),(282,366),(307,426),(269,443)],
    [(562,238),(684,167),(757,285),(634,357)],
    [(312,497),(792,475),(805,593),(318,612)],
    [(278,147),(447,144),(441,284),(256,287)],
    [(372,324),(585,373),(558,493),(328,435)],
    [(621,146),(741,139),(775,300),(639,305)]
]

In [15]:
from shapely import Polygon

intersections = []
unions = []
IOUs = []

for i in range(len(pred_boxes)):
    pred = Polygon(pred_boxes[i])
    GT = Polygon(GT_boxes[i])
    intersections.append(pred.intersection(GT).area)
    unions.append(pred.union(GT).area)
    IOUs.append(intersections[i] / unions[i])

print(intersections)
print(unions)
print(IOUs)

[28768.50373121497, 11190.0, 2718.0, 19167.5, 47205.7470914867, 24643.6161692114, 27475.29842483123, 20593.572699097153]
[28877.496268785035, 11779.5, 3119.5, 19700.0, 57500.25290851328, 25221.3838307886, 28877.20157516877, 21196.427300902844]
[0.9962256929563564, 0.9499554310454603, 0.8712934765186728, 0.9729695431472081, 0.8209659036907921, 0.9770921506348157, 0.9514529430184461, 0.9715586691451529]


In [ ]:
df["IOU"] = IOUs
print(df[["Picture", "Object", "IOU"]].to_markdown(index=False))

| Picture            | Object   |      IOU |
|:-------------------|:---------|---------:|
| Straight_no_shadow | Card     | 0.996226 |
| Straight_no_shadow | Circle   | 0.949955 |
| Straight_no_shadow | Complex  | 0.871293 |
| Straight_no_shadow | Square   | 0.97297  |
| Straight_no_shadow | Complex  | 0.820966 |
| Tilted_cards       | Square   | 0.977092 |
| Tilted_cards       | Card     | 0.951453 |
| Tilted_cards       | Card     | 0.971559 |

In [17]:
import numpy as np

print("Average IOU: ", np.sum(IOUs) / len(IOUs))

Average IOU:  0.938939226269613


# Measurements evaluation

In [36]:
pred_l = [5.3, 4.4, 3.2, 5.7, 3.7, 7.5, 8.7, 5.3]
GT_l = [5.3, 3.7, 2.3, 5, 4.1, 7.5, 8.7, 5.3]

pred_w = [8.5, 3.9, 1.9, 5.7, 19.5, 7.5, 5.1, 8.6]
GT_w = [8.4, 3.7, 1.3, 5, 19.3, 7.5, 5, 8.4]

In [38]:
from sklearn.metrics import mean_absolute_error

MAE_l = mean_absolute_error(pred_l,GT_l)
MAE_w = mean_absolute_error(pred_w,GT_w)

MAE = (MAEs_l + MAEs_w)/2

print("MAE length", MAE_l)
print("MAE width", MAE_w)
print("MAE", MAE)

MAE length 0.3375
MAE width 0.26249999999999973
MAE 0.2999999999999999


In [39]:
from sklearn.metrics import r2_score

r2 = r2_score(GT_l + GT_w, pred_l + pred_w)

print("R Squared", r2)

R Squared 0.9883741334354881
